In [14]:
import PyPDF2

# PDF 파일 경로 설정
pdf_file_path = 'C:/labeling/155_1_74123360_20070227_131900_1.pdf'

# PDF 파일 열기
with open(pdf_file_path, 'rb') as file:
    # PDF 리더 객체 생성
    pdf_reader = PyPDF2.PdfReader(file)
    
    # PDF 페이지 수 확인
    num_pages = len(pdf_reader.pages)
    
    # 모든 페이지의 텍스트 추출
    extracted_text = ""
    for page_num in range(num_pages):
        page = pdf_reader.pages[page_num]
        extracted_text += page.extract_text()
    
    # 추출된 텍스트 출력
    print(extracted_text)


Patient: CHO, SEONGWOO
ID: 74123360
Event Number Date/Time
Patient Event 1 of 1 13:19:00 27-Feb-2007
New Shape 1 of 346 13:19:00 27-Feb-2007
New Shape 2 of 346 13:19:03 27-Feb-2007
New Shape 3 of 346 13:19:03 27-Feb-2007
New Shape 4 of 346 13:19:04 27-Feb-2007
Noise 1 of 210 13:19:06 27-Feb-2007
Noise 2 of 210 13:19:10 27-Feb-2007
New Shape 5 of 346 13:20:02 27-Feb-2007
Noise 3 of 210 13:20:47 27-Feb-2007
New Shape 6 of 346 13:22:09 27-Feb-2007
Bradycardia 1 of 457 13:22:10 27-Feb-2007
New Shape 7 of 346 13:22:11 27-Feb-2007
New Shape 8 of 346 13:22:45 27-Feb-2007
New Shape 9 of 346 13:23:44 27-Feb-2007
New Shape 10 of 346 13:24:03 27-Feb-2007
New Shape 11 of 346 13:24:36 27-Feb-2007
New Shape 12 of 346 13:24:37 27-Feb-2007
Noise 4 of 210 13:25:04 27-Feb-2007
New Shape 13 of 346 13:25:55 27-Feb-2007
New Shape 14 of 346 13:26:17 27-Feb-2007
New Shape 15 of 346 13:26:18 27-Feb-2007
New Shape 16 of 346 13:26:20 27-Feb-2007
Noise 5 of 210 13:27:29 27-Feb-2007
Noise 6 of 210 13:27:33 27-Feb

In [15]:
import PyPDF2
import csv
import re
import os
from datetime import datetime
from tqdm import tqdm

# 파일이 있는 디렉토리 경로
directory_path = 'C:/labeling/'
output_directory = 'C:/labeling/output/'

# 환자별 CSV 파일을 저장할 디렉토리 생성
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# 모든 파일을 반복하며 환자별로 데이터 처리
files = sorted(os.listdir(directory_path))
current_patient_id = ""
event_data = []

# tqdm을 사용하여 파일 목록에 대한 진행 상태 표시
for file_name in tqdm(files, desc="Processing PDF files"):
    if file_name.endswith('.pdf'):
        # 파일명에서 환자 ID 추출 (예: 155_10)
        patient_id = "_".join(file_name.split('_')[:5])

        # 새로운 환자라면 이전 데이터를 CSV로 저장
        if current_patient_id and current_patient_id != patient_id:
            output_csv_path = f'{output_directory}{current_patient_id}.csv'
            with open(output_csv_path, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(["Event", "Number", "Time", "Date"])
                writer.writerows(event_data)
            event_data = []  # 이벤트 데이터를 초기화

        current_patient_id = patient_id

        # PDF 파일 열기
        pdf_file_path = os.path.join(directory_path, file_name)
        with open(pdf_file_path, 'rb') as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            num_pages = len(pdf_reader.pages)
            extracted_text = ""
            
            # 각 페이지에 대한 진행 상태 표시
            for page_num in tqdm(range(num_pages), desc=f"Extracting text from {file_name}", leave=False):
                page = pdf_reader.pages[page_num]
                extracted_text += page.extract_text()

        # 텍스트에서 이벤트 데이터 추출 (이벤트 종류에 상관없이)
        pattern = re.compile(r'([\w\s]+)\s(\d+ of \d+)\s([\d:]+)\s(\d{2}\-\w{3}\-\d{4})')
        matches = pattern.findall(extracted_text)

        # 날짜 형식 변경 및 데이터 추가
        for match in matches:
            event = match[0].strip()  # 이벤트 이름을 추출하고, 앞뒤 공백 제거
            number = match[1]
            time = match[2]
            date_str = match[3]

            # 날짜 형식을 '2007-03-01' 형식으로 변환
            date_obj = datetime.strptime(date_str, "%d-%b-%Y")
            formatted_date = date_obj.strftime("%Y-%m-%d")

            event_data.append([event, number, time, formatted_date])

# 마지막 환자에 대한 데이터도 저장
if event_data:
    output_csv_path = f'{output_directory}{current_patient_id}.csv'
    with open(output_csv_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Event", "Number", "Time", "Date"])
        writer.writerows(event_data)

print(f"모든 환자에 대한 CSV 파일이 {output_directory} 경로에 저장되었습니다.")


Processing PDF files: 100%|██████████| 186/186 [00:28<00:00,  6.45it/s]

모든 환자에 대한 CSV 파일이 C:/labeling/output/ 경로에 저장되었습니다.


In [16]:
import os
import pandas as pd
from tqdm import tqdm

# CSV 파일이 저장된 디렉토리 경로
output_directory = 'C:/labeling/output/'

# tqdm을 사용하여 파일 목록에 대한 진행 상태 표시
files = sorted(os.listdir(output_directory))

for file_name in tqdm(files, desc="Processing CSV files"):
    if file_name.endswith('.csv'):
        # CSV 파일 경로
        csv_file_path = os.path.join(output_directory, file_name)
        
        # CSV 파일 읽기
        df = pd.read_csv(csv_file_path)
        
        # "Time" 문자열이 포함된 행 제거
        df_cleaned = df[~df['Event'].str.contains("^Time", case=False)]
        
        # 깨끗한 데이터로 CSV 파일 덮어쓰기
        df_cleaned.to_csv(csv_file_path, index=False)

print(f"모든 CSV 파일이 {output_directory} 경로에서 수정되었습니다.")


Processing CSV files: 100%|██████████| 20/20 [00:00<00:00, 51.23it/s]

모든 CSV 파일이 C:/labeling/output/ 경로에서 수정되었습니다.
